In [1]:
import requests
import json
import pandas as pd
import collections
import logging

## List the URL 

### Get the time every 15 mins 

In [1]:
from datetime import datetime, timedelta

In [2]:
def get_time_string_list(start,end,interval):
    start = datetime.strptime(start, "%Y-%m-%d-%H-%M-%S")
    end = datetime.strptime(end, "%Y-%m-%d-%H-%M-%S")
    now = start
    string_list = []
    while now <= end:
        now += timedelta(minutes=15)
        string_list.append(now.strftime("%Y-%m-%d-%H-%M-%S"))
    return string_list

#time_list = get_time_string_list("2019-08-04-23-45-00","2019-12-08-23-45-00",15)
#print(len(time_list))

### URL of the data among all sites every 15 mins 

In [3]:
def get_url_list_all_data(time_list):
    url_list = []
    for i in range(len(time_list)):
        if i < len(time_list) - 1:
            url_time_start = time_list[i].split('-')
            url_day_start = url_time_start[0] + '-' + url_time_start[1] + '-' + url_time_start[2] + 'T'
            url_second_start = url_time_start[3] + '%3A' + url_time_start[4] + '%3A00Z'

            url_time_end = time_list[i+1].split('-')
            url_day_end = url_time_end[0] + '-' + url_time_end[1] + '-' + url_time_end[2] + 'T'
            url_second_end = url_time_end[3] + '%3A' + url_time_end[4] + '%3A00Z'    

            url_start = '&start=' + url_day_start + url_second_start
            url_end = '&end=' + url_day_end + url_second_end
            url = 'https://gcc.azure-api.net/traffic/v1/movement/history?size=1500' + url_start + url_end

            #print(url)
            url_list.append(url)

    return url_list

### URL of every site during whole research period  

In [14]:
def get_url_list_whole_period(sites_list):
    url_list_all_sites = []
    for site in sites_list:
        url = 'https://gcc.azure-api.net/traffic/v1/movement/history?size=90000&site=' + site + '&start=2023-03-01T00%3A00%3A00Z&end=2023-09-01T00%3A00%3A00Z'
        url_list_all_sites.append(url)

    return url_list_all_sites

## Get data from URL 

In [5]:
def get_data(url):
    response = requests.get(url)
    data = response.json()
    return data

## Reconstruct the data 

In [6]:
def construct_data(data):
    all_point_list = []
    for index in range(len(data)):
        #print(data[index])
        each_point = {}
        for feature, feature_value in data[index].items():
            #print({feature: feature_value})
            if feature != 'site':
                feature_dict = {feature: feature_value}
                #print(feature_dict)
                each_point = {**each_point, **feature_dict}
            else:
                for site_feature, site_feature_value in data[index][feature].items():
                    #print({site_feature: site_feature_value})
                    if site_feature == 'from':
                        for from_feature, from_feature_value in data[index][feature]['from'].items():
                            from_feature_dict = {'origin' + from_feature.capitalize(): from_feature_value}
                            #print(from_feature_dict)
                            each_point = {**each_point, **from_feature_dict}
                    elif site_feature == 'to':
                        for to_feature, to_feature_value in data[index][feature]['to'].items():
                            to_feature_dict = {'destination' + to_feature.capitalize(): to_feature_value}
                            #print(to_feature_dict)
                            each_point = {**each_point, **to_feature_dict}
                    else:
                        site_feature_dict = {'site' + site_feature.capitalize(): site_feature_value}
                        #print(site_feature_dict)
                        each_point = {**each_point, **site_feature_dict}
                    #if site_feature != 'from' and site_feature != 'to':
                        #print({'site' + site_feature.capitalize(): site_feature_value})
        #print(each_point)
        all_point_list.append(each_point)

    all_point_dict = {}
    for feature in all_point_list[0]:
        all_point_dict[feature] = [each_point[feature] for each_point in all_point_list]
    
    all_point_df = pd.DataFrame.from_dict(all_point_dict)
    return all_point_df


## Clean the data

### Clean the data via coords 

In [7]:
def data_clean_coords(all_point_df):
    #clean and rearrange df
    simple_df = all_point_df.drop(columns = ['siteType', 'siteLastupdate', 'siteId', 'site_id', '_id'])
    new_cols = ['type', 'lastUpdate', 'timestamp', 'batchIdentifier', 'siteSiteid', 'id', 'originDescription', 'originLat', 'originLong', 'destinationDescription', 'destinationLat', 'destinationLong', 'flow', 'concentration']
    output_df = simple_df[new_cols]

    #drop the data with zero coords
    data_error = []
    data_error_index = []
    for i in range(output_df.shape[0]):
        if output_df['originLat'][i] == '0':
            ola_olo_dla_dlo = str(i) + '-' + output_df['originLat'][i] + '-' + output_df['originLong'][i] + '-' + output_df['destinationLat'][i] + '-' + output_df['destinationLong'][i]
            data_error.append(ola_olo_dla_dlo)
            data_error_index.append(i)
        elif output_df['originLong'][i] == '0':
            ola_olo_dla_dlo = str(i) + '-' + output_df['originLat'][i] + '-' + output_df['originLong'][i] + '-' + output_df['destinationLat'][i] + '-' + output_df['destinationLong'][i]
            data_error.append(ola_olo_dla_dlo)
            data_error_index.append(i)
        elif output_df['destinationLat'][i] == '0':
            ola_olo_dla_dlo = str(i) + '-' + output_df['originLat'][i] + '-' + output_df['originLong'][i] + '-' + output_df['destinationLat'][i] + '-' + output_df['destinationLong'][i]
            data_error.append(ola_olo_dla_dlo)
            data_error_index.append(i)
        elif output_df['destinationLong'][i] == '0':
            ola_olo_dla_dlo = str(i) + '-' + output_df['originLat'][i] + '-' + output_df['originLong'][i] + '-' + output_df['destinationLat'][i] + '-' + output_df['destinationLong'][i]
            data_error.append(ola_olo_dla_dlo)
            data_error_index.append(i)

    output_df_correct = output_df.drop(data_error_index)
    output_df_correct = output_df_correct.reset_index(drop=True)

    #drop the data with incorrect coords
    data_incorrect_index = []
    data_incorrect = []
    for i in range(output_df_correct.shape[0]):
        if output_df_correct['originLat'][i] < '55':
            ola_dla = str(i) + '-' + output_df_correct['originLat'][i] + '-' + output_df_correct['destinationLat'][i]
            data_incorrect_index.append(i)
            data_incorrect.append(ola_dla)
        elif output_df_correct['destinationLat'][i] < '55':
            ola_dla = str(i) + '-' + output_df_correct['originLat'][i] + '-' + output_df_correct['destinationLat'][i]
            data_incorrect_index.append(i)
            data_incorrect.append(ola_dla)

    output_df_perfect = output_df_correct.drop(data_incorrect_index)
    output_df_perfect = output_df_perfect.reset_index(drop=True)
    
    return output_df_perfect

In [10]:
#constructed_data = construct_data(data)

NameError: name 'data' is not defined

In [66]:
#data_clean_coords(constructed_data)

,type,lastUpdate,timestamp,batchIdentifier,siteSiteid,id,originDescription,originLat,originLong,destinationDescription,destinationLat,destinationLong,flow,concentration
0,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:10:12,4817d225-9ea8-4a15-a160-54e3de7602bf,GA5751_T,movement-3d4141a5-985d-431c-90a3-e5ead725b0a6,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,0,0
1,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:15:11,4817d225-9ea8-4a15-a160-54e3de7602bf,GB0351_T,movement-1da80a4a-f1a5-4474-8226-0c1e5db1080c,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,4,0
2,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:15:11,4817d225-9ea8-4a15-a160-54e3de7602bf,GA4551_T,movement-4f7678e2-7439-47ab-922d-3ef659b256d4,George Square east side (N. Fredrick Street),55.86123268267249,-4.248956214731358,George Square east side (N. Fredrick Street),55.86054138016148,-4.248916255633903,0,0
3,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:15:11,4817d225-9ea8-4a15-a160-54e3de7602bf,GH050A_S,movement-05059bf3-6044-4219-958e-f2b766f3d910,Gallowgate east to Alma St,55.85421786382005,-4.214533994439329,Gallowgate east to Alma St,55.854293782200436,-4.214809937159854,3,0
4,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:10:12,4817d225-9ea8-4a15-a160-54e3de7602bf,GE2151_V,movement-fb216ae2-19f1-4567-8386-49d50fd454a7,Springburn Road to Keppochhill Road,55.87751512516916,-4.2323276748869665,Springburn Road to Keppochhill Road,55.877432004962884,-4.232450801952621,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:15:11,4817d225-9ea8-4a15-a160-54e3de7602bf,GF0251_R,movement-43be9862-f310-4aed-b977-db5142f9bf6e,Royston Rd e/b,55.87857876835941,-4.195185168972769,Royston Rd e/b,55.87839021402277,-4.194167303027818,0,0
970,Location,2021-10-13T06:00:00.5834829Z,2021-07-20T13:50:11,4817d225-9ea8-4a15-a160-54e3de7602bf,GC152A_C,movement-910e1a76-917c-463a-830e-f10792d24d97,Dumbarton Road inbound to Bunhouse Rd,55.869799016061826,-4.294639442160442,Dumbarton Road inbound to Bunhouse Rd,55.869255453811626,-4.293424537058428,0,0
971,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:15:11,4817d225-9ea8-4a15-a160-54e3de7602bf,GH4501_G,movement-e9d30256-1e70-4451-8d7d-75225e43b6cc,M74 w/b off ramp to Fullarton Roundabout,55.83482311880788,-4.1696580363248446,M74 w/b off ramp to Fullarton Roundabout,55.83494562438461,-4.171373617863877,0,0
972,Location,2021-10-13T06:00:00.5834829Z,2021-10-13T06:15:11,4817d225-9ea8-4a15-a160-54e3de7602bf,GB1551_T,movement-c7378429-500f-4c9a-8f96-2fa4ae71973c,Camden Terrace,55.845897093956474,-4.250594159531877,Camden Terrace,55.84560870404568,-4.250146178756491,0,0


### Clean the data via timestamp 

In [12]:
#############
#can be deleted after test
# url = 'https://gcc.azure-api.net/traffic/v1/movement/history?size=90000&site=GA5751_T&start=&start=2023-03-01T00%3A00%3A00Z&end=2023-09-01T00%3A00%3A00Z'
# data = get_data(url)
# all_time_data = construct_data(data)
# all_time_data

,type,lastUpdate,batchIdentifier,siteType,siteSiteid,siteLastupdate,originDescription,originLat,originLong,destinationDescription,destinationLat,destinationLong,siteId,site_id,timestamp,flow,concentration,id,_id
0,Location,2023-03-01T00:00:00.4809096Z,4ba954cc-6ba8-494f-97bf-17a5f1648472,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-02-28T23:55:12,0,0,movement-31b93b71-866b-434a-a45f-1200516238fe,movement-31b93b71-866b-434a-a45f-1200516238fe
1,Location,2023-03-01T00:15:00.2294947Z,e1122874-36a1-4bdc-a00b-85d117345379,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-03-01T00:10:12,0,0,movement-ef3d7e43-e22b-488f-a2b3-aa703f1e9d24,movement-ef3d7e43-e22b-488f-a2b3-aa703f1e9d24
2,Location,2023-03-01T00:30:00.2964527Z,b3bd1bbd-7b34-4879-8c23-ed478de98267,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-03-01T00:25:12,0,0,movement-5279a38b-c0ce-467b-bd09-eb3890d5bfd3,movement-5279a38b-c0ce-467b-bd09-eb3890d5bfd3
3,Location,2023-03-01T00:45:00.3537171Z,f06cf5f6-b9d9-4a52-be18-a9f32ad0a03f,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-03-01T00:40:12,0,0,movement-817ac85b-b47d-47ab-a966-64be53a6ed6c,movement-817ac85b-b47d-47ab-a966-64be53a6ed6c
4,Location,2023-03-01T01:00:00.4959901Z,1229b1c7-7531-4233-b38d-a88a4c8b9436,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-03-01T00:55:12,1,0,movement-80d3b1c9-7f37-44c6-a6ca-8efb8dd20327,movement-80d3b1c9-7f37-44c6-a6ca-8efb8dd20327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17624,Location,2023-08-31T22:45:00.337988Z,302ba06b-4a07-442d-ab49-d4ddfe9789fa,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-08-31T23:40:11,0,0,movement-566993b5-9af8-4578-8465-2cebc01ec16d,movement-566993b5-9af8-4578-8465-2cebc01ec16d
17625,Location,2023-08-31T23:00:01.947441Z,0f5ff82c-428e-4838-bea0-b9c6f9595444,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-08-31T23:55:11,0,0,movement-0aba71c4-fd16-4366-b3d5-1ee1805d9dfe,movement-0aba71c4-fd16-4366-b3d5-1ee1805d9dfe
17626,Location,2023-08-31T23:15:00.907561Z,6295ec55-d8ac-4977-b6af-fd06da078fb3,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-09-01T00:10:11,1,0,movement-f835ca5c-0b21-4da2-806f-7c8a6c49a076,movement-f835ca5c-0b21-4da2-806f-7c8a6c49a076
17627,Location,2023-08-31T23:30:00.9755636Z,efbb0aee-bdd8-4cf0-916a-76aa135c35cb,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-09-01T00:25:11,0,0,movement-e40044a2-3e33-41bc-9e6f-f5ff0967ba53,movement-e40044a2-3e33-41bc-9e6f-f5ff0967ba53


In [8]:
def data_unique_time(all_time_data):
    unique_time_dict = {}
    all_time_list = all_time_data['timestamp']
    date_counter = dict(collections.Counter(all_time_list))

    time_list = []
    count_list = []
    for time, count in date_counter.items():
        time_list.append(time)
        count_list.append(count)
    unique_time_dict['timestamp'] = time_list
    unique_time_dict['count'] = count_list

    unique_time_df = pd.DataFrame.from_dict(unique_time_dict)

    #unique the all_time_data dataframe via timestamp
    index = list(all_time_data.timestamp.drop_duplicates().index)
    unique_time_data = all_time_data.iloc[index]
    perfect_time_df = unique_time_data.merge(unique_time_df, how='left', left_on='timestamp', right_on='timestamp')

    #clean and rearrange df
    simple_df = perfect_time_df.drop(columns = ['siteType', 'siteLastupdate', 'siteId', 'site_id', '_id'])
    new_cols = ['lastUpdate', 'timestamp', 'siteSiteid', 'count', 'originDescription', 'originLat', 'originLong', 'destinationDescription', 'destinationLat', 'destinationLong', 'flow', 'concentration', 'batchIdentifier', 'id', 'type']
    output_perfect_df = simple_df[new_cols]

    return output_perfect_df

In [51]:
#data_unique_time(all_time_data)

### Convert coords WGS84 to OSGB36 

In [9]:
from convertbng.util import convert_bng

def convert_coords(great_time_df):
    ola = [float(i) for i in list(great_time_df['originLat'])]
    olo = [float(i) for i in list(great_time_df['originLong'])]
    dla = [float(i) for i in list(great_time_df['destinationLat'])]
    dlo = [float(i) for i in list(great_time_df['destinationLong'])]

    o_OSGB36 = convert_bng(olo, ola)
    d_OSGB36 = convert_bng(dlo, dla)
    o_easting = o_OSGB36[0]
    o_northing = o_OSGB36[1]
    d_easting = d_OSGB36[0]
    d_northing = d_OSGB36[1]

    great_time_df['originEasting'] = o_easting
    great_time_df['originNorthing'] = o_northing
    great_time_df['destinationEasting'] = d_easting
    great_time_df['destinationNorthing'] = d_northing

    return great_time_df

## Instance 

### Download one interval data to get all the sites with correct coords 

In [10]:
# time_list = get_time_string_list("2023-06-01-06-00-00","2023-06-01-06-15-00",15)
# url_list = get_url_list_all_data(time_list)
# for url in url_list:
#     data = get_data(url)
#     #print(len(data))
#     all_point_df = construct_data(data)
#     all_sites_df = data_clean_coords(all_point_df)
# all_sites_list = list(all_sites_df['siteSiteid'])
# print(len(set(all_sites_list)))
# #all_sites_list    

974


### Download whole research preiod data with correct sites 

In [13]:
# with open("C:/Users/2672496L/OneDrive - University of Glasgow/UofG/Data/Data processing/20211028_SCOOT_Data/JupyterNotebook_CSV/log_files/2021_12_13_SCOOT_data_download.log") as log:
#     read = log.read()
# read_split = read.split('INFO')[1:]
# sites_complete = []
# for i in read_split:
#     j = i.split('site=')[1].split('&')[0]
#     sites_complete.append(j)
# print(len(set(sites_complete)))
# #print(sites_complete)

# with open("C:/Users/2672496L/OneDrive - University of Glasgow/UofG/Data/Data processing/20211028_SCOOT_Data/JupyterNotebook_CSV/log_files/2021_12_13_v2_SCOOT_data_download.log") as log:
#     read = log.read()
# read_split = read.split('INFO')[1:]
# for i in read_split:
#     j = i.split('site=')[1].split('&')[0]
#     sites_complete.append(j)
# print(len(set(sites_complete)))

# remain_sites_list = [i for i in all_sites_list + sites_complete if i not in all_sites_list or i not in sites_complete]
# print(len(set(remain_sites_list)))
# remain_sites_list

97
192
782


['GB0401_V',
 'GL0521_R',
 'GH1021_S',
 'GC095A_T',
 'GD0601_D',
 'GH3171_C',
 'GH3171_G',
 'GC1351_A',
 'GJ3351_V',
 'GC180A_A',
 'GF2251_R',
 'GD210A_A',
 'GL1251_Y',
 'GL1701_A',
 'GK0051_C',
 'GE222A_B',
 'GA0571_N',
 'GA4651_S',
 'GB1301_V',
 'GC4151_C',
 'GD2701_A',
 'GG3001_A',
 'GD2301_A',
 'GC1601_P',
 'GA1361_B',
 'GF205A_T',
 'GA1451_S',
 'GK0401_C',
 'GA0651_D',
 'GJ0351_B',
 'GA2301_E',
 'GC095A_V',
 'GA4971_T',
 'GA5371_D',
 'GA0151_A',
 'GL1401_V',
 'GG268A_R',
 'GD0501_B',
 'GA1251_V',
 'GG0401_T',
 'GH3251_W',
 'GJ0151_D',
 'GJ0151_H',
 'GA4801_D',
 'GG0951_B',
 'GA5201_D',
 'GG267A_A',
 'GA1301_A',
 'GB1301_R',
 'GB1301_T',
 'GD2701_B',
 'GJ0051_D',
 'GA1601_A',
 'GG272A_S',
 'GH3451_N',
 'GA1201_D',
 'GK0251_V',
 'GJ0351_D',
 'GC4051_D',
 'GJ3201_V',
 'GK4601_T',
 'GE1801_G',
 'GB0251_S',
 'GD2451_A',
 'GA2301_C',
 'GH1021_R',
 'GH2701_G',
 'GA3221_B',
 'GH1051_C',
 'GA5701_S',
 'GA5371_F',
 'GA0301_W',
 'GA4751_C',
 'GE2291_S',
 'GB0601_C',
 'GA4351_S',
 'GJ2601_P',

In [14]:
# url_list_whole_period = get_url_list_whole_period(remain_sites_list)
# for url in url_list_whole_period:
#     data = get_data(url)
#     all_time_data = construct_data(data)
#     great_time_df = data_unique_time(all_time_data)
#     output_df = convert_coords(great_time_df)
    
#     csv_name = url.split('&', 2)[1].split('=')[1]
#     output_df.to_csv('C:/Users/2672496L/OneDrive - University of Glasgow/UofG/Data/Data processing/20211028_SCOOT_Data/JupyterNotebook_CSV/20211213_SCOOT_data/' + csv_name + '&2019_08_05-2021_12_12.csv', index = False)
#     #output_df.to_csv('20211213_SCOOT_data/' + csv_name + '&2019_08_05-2021_12_12.csv', index = False)
    
#     logging.basicConfig(filename='C:/Users/2672496L/OneDrive - University of Glasgow/UofG/Data/Data processing/20211028_SCOOT_Data/JupyterNotebook_CSV/log_files/2021_12_27_SCOOT_data_download.log', level=logging.INFO)
#     now = datetime.now()
#     current_time = now.strftime("%Y-%m-%d %H:%M:%S")
#     logging.info(current_time + '  ' + url)

KeyError: 0

### Get the timestamp with counting over 1 

In [24]:
# #test

# def get_url_list_whole_period(sites_list):
#     url_list_all_sites = []
#     for site in sites_list:
#         url = 'https://api.glasgow.gov.uk/traffic/v1/movement/history?size=90000&site=' + site + '&start=2023-01-05T00%3A00%3A00Z&end=2023-01-16T00%3A00%3A00Z'
#         url_list_all_sites.append(url)

#     return url_list_all_sites

In [25]:
# #test

# url_list_whole_period = get_url_list_whole_period(all_sites_list)
# for url in url_list_whole_period:
#     print(url)
#     data = get_data(url)
#     all_time_data = construct_data(data)
    
#     display(all_time_data)    

https://api.glasgow.gov.uk/traffic/v1/movement/history?size=90000&site=GA5751_T&start=2023-01-05T00%3A00%3A00Z&end=2023-01-16T00%3A00%3A00Z


,type,lastUpdate,batchIdentifier,siteType,siteSiteid,siteLastupdate,originDescription,originLat,originLong,destinationDescription,destinationLat,destinationLong,siteId,site_id,timestamp,flow,concentration,id,_id
0,Location,2023-01-05T00:00:00.4120571Z,12f0fd37-b216-44ef-b7a2-2b6d92bc219a,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-04T23:55:12,0,0,movement-906572ad-a408-42ab-b53b-63306f0d747a,movement-906572ad-a408-42ab-b53b-63306f0d747a
1,Location,2023-01-05T00:15:00.1711427Z,bfc198c6-985e-4642-8c1b-5cc5d45851a6,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-05T00:10:12,0,0,movement-65c06f86-1792-4847-8e5a-b1065b7793a9,movement-65c06f86-1792-4847-8e5a-b1065b7793a9
2,Location,2023-01-05T00:30:00.1734951Z,10de5d17-2148-4aff-8c52-1cadf19e5558,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-05T00:25:12,0,0,movement-054a13e7-2776-4f62-bd25-b71f5cfd2943,movement-054a13e7-2776-4f62-bd25-b71f5cfd2943
3,Location,2023-01-05T00:45:00.4145305Z,8846332a-d383-4999-a390-e904d7696469,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-05T00:40:12,0,0,movement-5afe0bb2-ba97-4ce4-a36f-c674e81adb74,movement-5afe0bb2-ba97-4ce4-a36f-c674e81adb74
4,Location,2023-01-05T01:00:00.2291664Z,2fa6621b-8367-40fe-b045-278afb756b0d,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-05T00:55:12,0,0,movement-95a55b9d-dce0-4250-a826-97eee3851e07,movement-95a55b9d-dce0-4250-a826-97eee3851e07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,Location,2023-01-15T22:45:00.436895Z,b67a5cc4-0bdf-46f9-94d6-84d28f3b3a3a,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-15T22:40:12,0,0,movement-8893f11d-2432-42a6-ac80-6f7c09729b67,movement-8893f11d-2432-42a6-ac80-6f7c09729b67
1052,Location,2023-01-15T23:00:00.403254Z,3e475781-5af3-401d-9d18-f352ddc2c8ef,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-15T22:55:12,0,0,movement-47a7a0e6-49fc-4c19-ad7d-86bb1beb150e,movement-47a7a0e6-49fc-4c19-ad7d-86bb1beb150e
1053,Location,2023-01-15T23:15:00.2173934Z,4cd7aeeb-c515-4af7-8a5e-2c04b3744788,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-15T23:10:12,0,0,movement-2f7c2bc5-2801-4cb2-80cc-112fcaf27194,movement-2f7c2bc5-2801-4cb2-80cc-112fcaf27194
1054,Location,2023-01-15T23:30:00.6037039Z,f45299f5-8cfb-4b5c-ac97-d44b3d686377,Location,GA5751_T,0001-01-01T00:00:00,Pinkston Road,55.86923861751409,-4.238709782908749,Pinkston Road,55.8690661389906,-4.235838725498642,site-GA5751_T,site-GA5751_T,2023-01-15T23:25:12,0,0,movement-6e14e0f9-f62f-496f-82d8-774f8f647b8e,movement-6e14e0f9-f62f-496f-82d8-774f8f647b8e


https://api.glasgow.gov.uk/traffic/v1/movement/history?size=90000&site=GB0351_T&start=2023-01-05T00%3A00%3A00Z&end=2023-01-16T00%3A00%3A00Z


,type,lastUpdate,batchIdentifier,siteType,siteSiteid,siteLastupdate,originDescription,originLat,originLong,destinationDescription,destinationLat,destinationLong,siteId,site_id,timestamp,flow,concentration,id,_id
0,Location,2023-01-05T00:00:00.4120571Z,12f0fd37-b216-44ef-b7a2-2b6d92bc219a,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-04T23:55:10,4,0,movement-d05753d9-7e8d-4286-ae02-5cb16add7393,movement-d05753d9-7e8d-4286-ae02-5cb16add7393
1,Location,2023-01-05T00:15:00.1711427Z,bfc198c6-985e-4642-8c1b-5cc5d45851a6,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-05T00:10:10,4,0,movement-88353e5e-5066-44fd-a311-65d08df9c704,movement-88353e5e-5066-44fd-a311-65d08df9c704
2,Location,2023-01-05T00:30:00.1734951Z,10de5d17-2148-4aff-8c52-1cadf19e5558,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-05T00:25:10,4,0,movement-a66b126a-5db5-4038-a40c-7da18916506e,movement-a66b126a-5db5-4038-a40c-7da18916506e
3,Location,2023-01-05T00:45:00.4145305Z,8846332a-d383-4999-a390-e904d7696469,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-05T00:40:10,4,0,movement-23b8f379-a050-40a0-a871-db7d8edd1694,movement-23b8f379-a050-40a0-a871-db7d8edd1694
4,Location,2023-01-05T01:00:00.2291664Z,2fa6621b-8367-40fe-b045-278afb756b0d,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-05T00:55:10,4,0,movement-68f1ba60-0149-423a-a480-a5c562e5f3b2,movement-68f1ba60-0149-423a-a480-a5c562e5f3b2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,Location,2023-01-15T22:45:00.436895Z,b67a5cc4-0bdf-46f9-94d6-84d28f3b3a3a,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-15T22:40:11,0,0,movement-7090923a-fb30-417c-ada1-7a90a4d705a8,movement-7090923a-fb30-417c-ada1-7a90a4d705a8
1052,Location,2023-01-15T23:00:00.403254Z,3e475781-5af3-401d-9d18-f352ddc2c8ef,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-15T22:55:11,0,0,movement-11adbd12-4ff3-48ef-a22a-5a8218f62f42,movement-11adbd12-4ff3-48ef-a22a-5a8218f62f42
1053,Location,2023-01-15T23:15:00.2173934Z,4cd7aeeb-c515-4af7-8a5e-2c04b3744788,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-15T23:10:11,9,0,movement-f80dd05d-92c8-4050-91c7-f11e9e0182df,movement-f80dd05d-92c8-4050-91c7-f11e9e0182df
1054,Location,2023-01-15T23:30:00.6037039Z,f45299f5-8cfb-4b5c-ac97-d44b3d686377,Location,GB0351_T,0001-01-01T00:00:00,Tradeston Street to Cook Street southbound,55.85212573722932,-4.263799903239596,Tradeston Street to Cook Street southbound,55.852191225829486,-4.263659918180315,site-GB0351_T,site-GB0351_T,2023-01-15T23:25:11,9,0,movement-3

https://api.glasgow.gov.uk/traffic/v1/movement/history?size=90000&site=GA4551_T&start=2023-01-05T00%3A00%3A00Z&end=2023-01-16T00%3A00%3A00Z


...

,type,lastUpdate,batchIdentifier,siteType,siteSiteid,siteLastupdate,originDescription,originLat,originLong,destinationDescription,destinationLat,destinationLong,siteId,site_id,timestamp,flow,concentration,id,_id
0,Location,2023-01-05T00:00:00.4120571Z,12f0fd37-b216-44ef-b7a2-2b6d92bc219a,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-04T23:55:10,3,0,movement-9387a500-1661-47bd-9656-cf4f6653bfe5,movement-9387a500-1661-47bd-9656-cf4f6653bfe5
1,Location,2023-01-05T00:15:00.1711427Z,bfc198c6-985e-4642-8c1b-5cc5d45851a6,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-05T00:10:10,4,0,movement-d37f4d4e-84cc-46df-9896-c0d4e5fa2aa7,movement-d37f4d4e-84cc-46df-9896-c0d4e5fa2aa7
2,Location,2023-01-05T00:30:00.1734951Z,10de5d17-2148-4aff-8c52-1cadf19e5558,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-05T00:25:10,7,0,movement-4b0b67c0-0717-491e-a4ba-5651bae07742,movement-4b0b67c0-0717-491e-a4ba-5651bae07742
3,Location,2023-01-05T00:45:00.4145305Z,8846332a-d383-4999-a390-e904d7696469,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-05T00:40:10,1,0,movement-75a4027b-353e-4b74-951c-e8a528067459,movement-75a4027b-353e-4b74-951c-e8a528067459
4,Location,2023-01-05T01:00:00.2291664Z,2fa6621b-8367-40fe-b045-278afb756b0d,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-05T00:55:10,2,0,movement-4d0916a7-0201-493e-8505-0b365eaeae78,movement-4d0916a7-0201-493e-8505-0b365eaeae78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,Location,2023-01-15T22:45:00.436895Z,b67a5cc4-0bdf-46f9-94d6-84d28f3b3a3a,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-15T22:40:11,4,0,movement-a736df4d-9a13-4940-ab44-1848d64bba9e,movement-a736df4d-9a13-4940-ab44-1848d64bba9e
1052,Location,2023-01-15T23:00:00.403254Z,3e475781-5af3-401d-9d18-f352ddc2c8ef,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-15T22:55:11,3,0,movement-dab1f475-79e3-459f-94d9-54a62aedc87b,movement-dab1f475-79e3-459f-94d9-54a62aedc87b
1053,Location,2023-01-15T23:15:00.2173934Z,4cd7aeeb-c515-4af7-8a5e-2c04b3744788,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-15T23:10:11,4,0,movement-aab3c8c1-6bc5-40dc-8bf4-41832b498125,movement-aab3c8c1-6bc5-40dc-8bf4-41832b498125
1054,Location,2023-01-15T23:30:00.6037039Z,f45299f5-8cfb-4b5c-ac97-d44b3d686377,Location,GA5471_D,0001-01-01T00:00:00,Motorway off ramp Castle St. southbound,55.865429441247834,-4.236157121242562,Motorway off ramp Castle St. southbound,55.8648959825708,-4.235838776125701,site-GA5471_D,site-GA5471_D,2023-01-15T23:25:11,0,0,movement-903f1548-6121-43fc-ba4a-ca298bc473a9,movement-903f1548-6121-43fc

https://api.glasgow.gov.uk/traffic/v1/movement/history?size=90000&site=GJ3201_R&start=2023-01-05T00%3A00%3A00Z&end=2023-01-16T00%3A00%3A00Z


,type,lastUpdate,batchIdentifier,siteType,siteSiteid,siteLastupdate,originDescription,originLat,originLong,destinationDescription,destinationLat,destinationLong,siteId,site_id,timestamp,flow,concentration,id,_id
0,Location,2023-01-05T00:00:00.4120571Z,12f0fd37-b216-44ef-b7a2-2b6d92bc219a,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-04T23:55:11,0,0,movement-e567b47d-fa78-4e95-a751-3795fc96e1d1,movement-e567b47d-fa78-4e95-a751-3795fc96e1d1
1,Location,2023-01-05T00:15:00.1711427Z,bfc198c6-985e-4642-8c1b-5cc5d45851a6,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-05T00:10:11,0,0,movement-a2f7387a-c3e4-43bf-96d2-d47b6217531b,movement-a2f7387a-c3e4-43bf-96d2-d47b6217531b
2,Location,2023-01-05T00:30:00.1734951Z,10de5d17-2148-4aff-8c52-1cadf19e5558,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-05T00:25:11,0,0,movement-844b5ef0-4504-4ef9-ba0f-ae4611a76744,movement-844b5ef0-4504-4ef9-ba0f-ae4611a76744
3,Location,2023-01-05T00:45:00.4145305Z,8846332a-d383-4999-a390-e904d7696469,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-05T00:40:11,0,0,movement-d1961bce-2f83-47e5-9797-6242341e2ccc,movement-d1961bce-2f83-47e5-9797-6242341e2ccc
4,Location,2023-01-05T01:00:00.2291664Z,2fa6621b-8367-40fe-b045-278afb756b0d,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-05T00:55:11,0,0,movement-f869422b-38d0-400a-b193-357e2eeb3181,movement-f869422b-38d0-400a-b193-357e2eeb3181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,Location,2023-01-15T22:45:00.436895Z,b67a5cc4-0bdf-46f9-94d6-84d28f3b3a3a,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-15T22:40:12,0,0,movement-ea7f446d-355c-4722-b05a-cb17f7c37031,movement-ea7f446d-355c-4722-b05a-cb17f7c37031
1052,Location,2023-01-15T23:00:00.403254Z,3e475781-5af3-401d-9d18-f352ddc2c8ef,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-15T22:55:12,0,0,movement-eda2a950-4d9b-48a0-b621-bb6dd8c2e173,movement-eda2a950-4d9b-48a0-b621-bb6dd8c2e173
1053,Location,2023-01-15T23:15:00.2173934Z,4cd7aeeb-c515-4af7-8a5e-2c04b3744788,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-15T23:10:11,0,0,movement-7f0b8305-3f5b-439e-960e-67c62ddbe678,movement-7f0b8305-3f5b-439e-960e-67c62ddbe678
1054,Location,2023-01-15T23:30:00.6037039Z,f45299f5-8cfb-4b5c-ac97-d44b3d686377,Location,GJ3201_R,0001-01-01T00:00:00,Battlefield Rd,55.824415626053,-4.259491328371662,Battlefield Rd,55.82410688116023,-4.258196175251756,site-GJ3201_R,site-GJ3201_R,2023-01-15T23:25:12,0,0,movement-c19cbb2c-2f8d-4aae-9cbd-3c551268db26,movement-c19cbb2c-2f8d-4aae-9cbd-3c551268db26


https://api.glasgow.gov.uk/traffic/v1/movement/history?size=90000&site=GA2151_S&start=2023-01-05T00%3A00%3A00Z&end=2023-01-16T00%3A00%3A00Z


KeyError: 0